# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096843


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:  10%|█         | 3/30 [00:07<01:07,  2.49s/it]

Rendering models:  17%|█▋        | 5/30 [00:08<00:45,  1.83s/it]

Rendering models:  20%|██        | 6/30 [00:08<00:34,  1.44s/it]

Rendering models:  23%|██▎       | 7/30 [00:09<00:30,  1.32s/it]

...

Rendering models:  90%|█████████ | 27/30 [00:21<00:01,  1.75it/s]

Rendering models:  93%|█████████▎| 28/30 [00:22<00:01,  1.72it/s]

Rendering models:  97%|█████████▋| 29/30 [00:22<00:00,  1.70it/s]

Rendering models: 100%|██████████| 30/30 [00:23<00:00,  1.87it/s]

not-logged-in-3b0b929f4475fd276561    0.002308
jnarayanbvg                           0.000590
awright5                              0.008550
deivad                                0.040944
claney99                              0.001487
not-logged-in-d78ab5a3057206e8280b    1.212846
framosuw                              0.000363
M4kc                                  0.000594
not-logged-in-a6e512f458a49544fc6d    0.001770
lsautter                              0.000417
not-logged-in-3bf20897ad3067a962a6    0.000568
earthling001                          0.000977
coyotes54                             0.000499
not-logged-in-9f30f411df9d28d2c541    0.000744
TRISTANM18                            0.000499
Lavadude                              0.008760
clars915                              0.000386
lgonzalez62                           0.002142
r.sundstrom3                          0.000836
not-logged-in-e9900be6759309fd7692    0.000504
jmacd297                              0.000734
h4ranek      

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())